In [1]:
import ast  # for converting embeddings saved as strings back to arrays
import openai # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
import os # for getting API token from env variable OPENAI_API_KEY
from scipy import spatial  # for calculating vector similarities for search

In [2]:
# Configuración de OpenAI
openai.api_key = "sk-proj-JEQe19qf1majBtkGMsubT3BlbkFJeGguQ7FTMjap1uVkFViu"
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"
MAX_TOKENS = 4096

In [3]:
def num_tokens(text: str, model: str = EMBEDDING_MODEL) -> int:
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [4]:
def create_embeddings_from_txt_files(directory: str, model: str = EMBEDDING_MODEL, max_tokens: int = 4096) -> pd.DataFrame:
    texts = []
    embeddings = []
    filenames = []
    
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r", encoding="utf-8") as file:
                text = file.read()
                
                # Split the text into manageable chunks
                token_count = num_tokens(text, model=model)
                if token_count > max_tokens:
                    print(f"Warning: File {filename} exceeds max token limit and will be truncated.")
                    text = text[:max_tokens]
                
                response = openai.Embedding.create(model=model, input=text)
                embedding = response['data'][0]['embedding']
                
                texts.append(text)
                embeddings.append(embedding)
                filenames.append(filename)
    
    df = pd.DataFrame({"text": texts, "embedding": embeddings, "filename": filenames})
    return df

In [6]:
# Uso de la función
directory = "data"
df = create_embeddings_from_txt_files(directory)
df.to_excel("vectores.xlsx", index=False)
df.to_csv("vectores.csv", index=False)
df

,text,embedding,filename
0,"El comando ""Enviar mensaje al conductor"" permi...","[-0.02398996241390705, 0.00521461758762598, -0...",Conductores.txt
1,\nEl dashboard en la plataforma de seguimiento...,"[-0.023207632824778557, -0.0013814468402415514...",Dashboard.txt
2,\nCreación y Gestión de Geocercas en el Sistem...,"[-0.013827535323798656, 0.02504660375416279, 0...",Geocercas.txt
3,\nLa plataforma de seguimiento ofrece diversas...,"[-0.02603936940431595, 0.01353693287819624, -0...",Herramientas.txt
4,"En la pestaña de informes, se pueden crear inf...","[-0.00831671804189682, 0.004125538282096386, 0...",Informes.txt
5,Introducción a la plataforma:\n\nExplicación d...,"[0.004063657484948635, 0.0053825052455067635, ...",Introducción a la plataforma.txt
6,\nGestión de Notificaciones en el Sistema de S...,"[-0.019962914288043976, 0.013457619585096836, ...",Notificaciones.txt
7,La pestaña de recorridos en la plataforma de s...,"[-0.012566705234348774, -0.012425732798874378,...",Recorridos.txt
8,"Además de la pestaña Remolques, se pueden ver ...","[-0.017275424674153328, 0.00032052703318186104...",Remolques.txt
9,Algunos tipos de dispositivos soportan los com...,"[-0.013809074647724628, 0.007888940162956715, ...",Rutas.txt


# Filtracion df

In [30]:
def display_and_select_files(directory: str) -> list:
    files = [f for f in os.listdir(directory) if f.endswith(".txt")]
    print("Archivos disponibles:")
    for i, file in enumerate(files, start=1):
        print(f"{i}. {file}")
    
    while True:
        try:
            selected_indices = input("Ingrese los números de los archivos que desea usar, separados por comas: ")
            selected_indices = [int(i.strip()) for i in selected_indices.split(",")]
            selected_files = [files[i - 1] for i in selected_indices]
            break
        except ValueError:
            print("Entrada inválida. Por favor, ingrese números separados por comas.")
        except IndexError:
            print("Uno o más números están fuera del rango. Por favor, inténtelo de nuevo.")
    
    return selected_files

def filter_dataframe_by_filenames(df: pd.DataFrame, filenames: list) -> pd.DataFrame:
    filtered_df = df[df["filename"].isin(filenames)].copy()
    filtered_df.drop(columns=["filename"], inplace=True)
    return filtered_df

In [31]:
# Ejemplo de uso
directory = "data"
df = create_embeddings_from_txt_files(directory)

selected_files = display_and_select_files(directory)
filtered_df = filter_dataframe_by_filenames(df, selected_files)
print(filtered_df.head())

Archivos disponibles:
1. Conductores y remolques.txt
2. DAshboard.txt
3. Geocercas.txt
4. Herramientas Parte 1.txt
5. Herramientas Parte 2.txt
6. Herramientas.txt
7. iDriveSafe.txt
8. Informes.txt
9. Introducción a la plataforma.txt
10. Notificaciones.txt
11. Panel de Informes lectura.txt
12. Panel de informes(Ejecución).txt
13. Panel Informes Creacion.txt
14. Recorridos.txt
15. Seguimiento.txt
16. Sensolator.txt
17. Tareas.txt
18. Trackplayer.txt
19. Unidades.txt
20. Usuarios.txt
                                                 text  \
2   \nCreación y Gestión de Geocercas en el Sistem...   
16  \nLa pestaña de tareas en la plataforma de seg...   

                                            embedding  
2   [-0.013827535323798656, 0.02504660375416279, 0...  
16  [-0.02037661150097847, -0.01038912683725357, -...  


In [32]:
# Función para clasificar las strings por relevancia usando embeddings
def strings_ranked_by_relatedness(query: str, df: pd.DataFrame, top_n: int = 5, threshold: float = 0.76) -> list[tuple[str, float]]:
    query_embedding_response = openai.Embedding.create(model=EMBEDDING_MODEL, input=query)
    query_embedding = query_embedding_response['data'][0]['embedding']
    strings_and_relatednesses = [(row["text"], 1 - spatial.distance.cosine(query_embedding, row["embedding"])) for _, row in df.iterrows()]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    valid_sections = [(s, r) for s, r in strings_and_relatednesses if r >= threshold]
    
    return valid_sections[:top_n]

In [33]:
# Función para crear el mensaje de consulta
def query_message(query: str, df: pd.DataFrame, model: str, token_budget: int, threshold: float = 0.76) -> str:
    valid_sections = strings_ranked_by_relatedness(query, df, threshold=threshold)
    
    if len(valid_sections) == 0:
        return "No cuento con esa información, por favor contactar a servicio a cliente."
    
    introduction = 'Use the below articles to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string, relatedness in valid_sections:
        next_article = f'\n\nText section:\n"""\n{string}\n"""'
        if num_tokens(message + next_article + question, model=model) > token_budget:
            break
        else:
            message += next_article
    return message + question

In [34]:
# Función para hacer la consulta a GPT
def ask(query: str, df: pd.DataFrame, model: str = GPT_MODEL, token_budget: int = 4096 - 500, print_message: bool = False, threshold: float = 0.76) -> str:
    message = query_message(query, df, model=model, token_budget=token_budget, threshold=threshold)
    if message == "No cuento con esa información, por favor contactar a servicio a cliente.":
        return message
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You answer questions based on the provided articles."},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(model=model, messages=messages, temperature=0)
    return response['choices'][0]['message']['content']

In [35]:
def print_formatted_response(response: str) -> None:
    formatted_response = response.replace("\\n", "\n")
    formatted_response = formatted_response.replace('Wialon', 'Quamtum')
    return formatted_response

In [37]:
# Ejemplo de uso
query = "Que es una tarea"
response = ask(query, filtered_df, model=GPT_MODEL, print_message=False, threshold=0.76)
print(print_formatted_response(response))

Una tarea es una acción automatizada que se puede programar en la plataforma de seguimiento para generar y enviar informes por correo electrónico, permitiendo configurar parámetros como la plantilla de informe, formato de envío, intervalos de informes, destinatarios, objeto del informe, nombre de la tarea, frecuencia de activación, fecha y hora de inicio, entre otros detalles. Una vez creada, la tarea se muestra en la lista de trabajo con su nombre, icono y descripción, y se pueden realizar acciones como activarla, desactivarla, ejecutarla como prueba, ver el número de ejecuciones, acceder a sus propiedades, copiarla o eliminarla.
